## Part 1

In [51]:
import scala.io.Source
import scala.collection.mutable.ListBuffer

val filename = "input"
val lines = Source.fromFile(filename).getLines.toList.map(line => line.split(" ")).map(strings => (strings(1), strings(7)))

import scala.io.Source
import scala.collection.mutable.ListBuffer
filename: String = "input"
lines: List[(String, String)] = List(
  ("R", "Y"),
  ("N", "T"),
  ("C", "L"),
  ("F", "B"),
  ("L", "D"),
  ("T", "D"),
  ("O", "E"),
  ("M", "Z"),
  ("A", "V"),
  ("K", "D"),
  ("W", "I"),
  ("B", "J"),
  ("H", "D"),
  ("P", "J"),
...

In [52]:
// These lines now represent dependencies - line(0) must be finished before line(1)
val dependents = lines.groupBy(x => x._2).map({case (task, deps) => task -> deps.map(x => x._1)})

dependents: Map[String, List[String]] = Map(
  "E" -> List(
    "O",
    "Y",
    "G",
    "D",
    "U",
    "I",
    "Q",
    "Z",
    "W",
    "P",
    "N"
  ),
  "X" -> List("S", "J", "Q", "Y", "Z"...

In [53]:
val allTasks = lines.flatMap(x => List(x._1, x._2)).toSet

allTasks: Set[String] = Set(
  "E",
  "X",
  "N",
  "T",
  "Y",
  "J",
  "U",
  "F",
  "A",
  "M",
  "I",
  "G",
  "V",
  "Q",
...

In [54]:
// Initialise the task list (ListBuffer)
var lettersToProcess = allTasks.toList.filter(task => !dependents.contains(task)).sorted.to[ListBuffer]
// Ensure the dependents list contains empty lists for the non-dependent tasks
val allDependents = dependents ++ lettersToProcess.map(letter => letter -> List())
var completedLetters = scala.collection.mutable.Set[String]()

lettersToProcess: ListBuffer[String] = ListBuffer("C", "F", "N", "R")
allDependents: Map[String, List[String]] = Map(
  "E" -> List(
    "O",
    "Y",
    "G",
    "D",
    "U",
    "I",
    "Q",
    "Z",
    "W",
    "P",
    "N"
  ),
  "X" -> List("S", "J", "Q", "Y", "Z"...
completedLetters: collection.mutable.Set[String] = Set()

In [55]:
while (!lettersToProcess.isEmpty) {
    val letter = lettersToProcess.head
    print(letter)
    completedLetters += letter
    // Recalculate lettersToProcess
    lettersToProcess = allDependents.filter({case (task, deps) => deps.forall(x => completedLetters.contains(x))})
        .map(x => x._1)
        .filter(task => !completedLetters.contains(task))
        .to[ListBuffer].sorted   
}

CFMNLOAHRKPTWBJSYZVGUQXIDE

## Part 2

In [56]:
// Build up the mapping of task -> numSeconds
val stepDuration = 60
val taskSecondsMap = allTasks.map(task => (task -> (task.head - 64 + stepDuration))).toMap

stepDuration: Int = 60
taskSecondsMap: Map[String, Int] = Map(
  "E" -> 65,
  "X" -> 84,
  "N" -> 74,
  "T" -> 80,
  "Y" -> 85,
  "J" -> 70,
  "U" -> 81,
  "F" -> 66,
  "A" -> 61,
  "M" -> 73,
  "I" -> 69,
  "G" -> 67,
  "V" -> 82,
  "Q" -> 77,
...

In [57]:
// Initialise the tasks that each worker is working on and the remaining seconds of the task
val numWorkers = 5
var workerTasks = Array.fill(numWorkers)(("", -1))

numWorkers: Int = 5
workerTasks: Array[(String, Int)] = Array(
  ("", -1),
  ("", -1),
  ("", -1),
  ("", -1),
  ("", -1)
)

In [58]:
// Set up the queues of potential tasks
var completedTasks = Set[String]()
var availableTasks = allDependents.filter({case (task, deps) => deps.forall(x => completedTasks.contains(x))})
    .map(x => x._1).to[ListBuffer].sorted
var unreachableTasks = allTasks.diff(availableTasks.toSet)

completedTasks: Set[String] = Set()
availableTasks: ListBuffer[String] = ListBuffer("C", "F", "N", "R")
unreachableTasks: Set[String] = Set(
  "E",
  "X",
  "T",
  "Y",
  "J",
  "U",
  "A",
  "M",
  "I",
  "G",
  "V",
  "Q",
  "L",
  "B",
...

In [59]:
// Function to assign a worker to a new task
def getNextTask() : (String, Int) = {
    println(availableTasks)
    if (availableTasks.length == 0) {
        println("Returning None task...")
        return (("", -1))
    }
    var task = availableTasks.remove(0)
    println("Task:", task)
    // Pop the next task off of the available tasks list and return it with its seconds
    return (task, taskSecondsMap(task))
}

defined function getNextTask

In [60]:
// Function to represent completion of a task (taskRemainingSeconds == 0)
def completeTask(task: String) : (String, Int) = {
    // Push the task on to the completed task list. 
    println("Task: ", task)
    completedTasks += task
    println("Completed tasks: ", completedTasks)
    // Recalculate the available tasks, popping any unreachable tasks from the set 
    availableTasks = unreachableTasks.union(availableTasks.toSet)
        .map(task => (task, allDependents(task)))
        .filter({case (task, deps) => deps.forall(x => completedTasks.contains(x))})
        .map(x => x._1)
        .to[ListBuffer].sorted
    unreachableTasks = unreachableTasks.diff(availableTasks.toSet)
    // Set the worker's current task to ("", -1)
    return (("", -1))
}

defined function completeTask

In [61]:
completedTasks == allTasks

res60: Boolean = false

In [62]:
// Loop to increment the seconds and call the relevant methods
var seconds = 0
while (completedTasks != allTasks) {
    seconds += 1
    println("Seconds elapsed:", seconds)
    // Iterate across each worker, updating the relevant queues
    for (workerID <- 0 to workerTasks.length-1) {
        println("Worker:", workerID)
        var currentTask = workerTasks(workerID)
        println("Worker existing task: ", currentTask)
        // If current task seconds have reached 0, complete the task
        if (currentTask._2 == 0) {
            //println("Completing task...")
            // Push the task on to the completed task list. 
            completedTasks += currentTask._1
            // Recalculate the available tasks, popping any unreachable tasks from the set 
            availableTasks = unreachableTasks.union(availableTasks.toSet)
                .map(task => (task, allDependents(task)))
                .filter({case (task, deps) => deps.forall(x => completedTasks.contains(x))})
                .map(x => x._1)
                .to[ListBuffer].sorted
            unreachableTasks = unreachableTasks.diff(availableTasks.toSet)
            // Set the worker's current task to ("", -1)
            currentTask = ("", -1)
        }
        // If no task, assign one
        if (currentTask._1 == "") {
            //println("Getting new task...")
            if (availableTasks.length == 0) {
                //println("Returning None task...")
                currentTask = (("", -1))
            }
            else {
                var task = availableTasks.remove(0)
                //println("Task:", task)
                // Pop the next task off of the available tasks list and return it with its seconds
                currentTask = (task, taskSecondsMap(task))
                //print("Returned task: ", getNextTask())
            }
        }
        // Decrement the seconds for the task and reassign to worker
        workerTasks(workerID) = (currentTask._1, currentTask._2-1)
        //println("Current task: ", currentTask)
    }
    //println("CT: ", completedTasks)
    //println("AT: ", availableTasks)
    //println("UT: ", unreachableTasks)
}
println("Seconds:", seconds-1)

(Seconds:,972)


seconds: Int = 973